# Urgency Detector

transformers → For working with DeepSeek models.
datasets → For handling urgency_data.csv efficiently.
peft → For LoRA fine-tuning (saves memory).
accelerate → Optimizes training performance.
bitsandbytes → Enables 8-bit and 4-bit quantization to reduce RAM usage.
torch → PyTorch for model training.

In [ ]:
pip install torch_xla[tpu] -f https://storage.googleapis.com/libtpu-releases/index.html

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch_xla
import torch_xla.core.xla_model as xm

# Load the smallest DeepSeek model
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # We'll replace this with a smaller model if needed

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load model (set device to TPU if available)
device = xm.xla_device()
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=6
).to(device)

# Verify model is loaded
print(f"Model loaded on {device}")

# Test Tokenization on Sample Data
Before fine-tuning, let’s verify that our tokenizer is working properly:

# Initial Model Testing on Small Dataset
Before fine-tuning, we want to check how well the base DeepSeek model classifies urgency without training.

1. Prepare a Few Sample Inputs for Testing
We will create a small dataset with a few labeled examples:

In [ ]:
sample_conversations = [
    {
        "conversation": (
            "Caller: I need you to send money immediately, I am in big trouble!\n"
            "Receiver: What happened? Why do you need money so urgently?\n"
            "Caller: I got into a legal issue, and I need bail money right now!"
        ),
        "label": "Legal/Authority Urgency"
    },
    {
        "conversation": (
            "Caller: Hey, I just saw this new phone, and it’s on a flash sale for 80% off! You have to buy it now!\n"
            "Receiver: That’s a crazy deal! How long is the sale?\n"
            "Caller: It’s only for the next 5 minutes! Hurry up!"
        ),
        "label": "Social/Peer Pressure Urgency"
    },
    {
        "conversation": (
            "Caller: The bank just called. They said there’s an issue with your account, and you need to verify your details now.\n"
            "Receiver: Wait, is this a scam? My bank never calls like this.\n"
            "Caller: No, this is serious! If you don’t confirm, your account will be frozen!"
        ),
        "label": "Financial Urgency"
    }
]

labels = ["Emotional Urgency", "Financial Urgency", "Legal/Authority Urgency", "No Urgency", "Social/Peer Pressure Urgency", "Romantic Urgency"]
label_to_id = {label: i for i, label in enumerate(labels)}


2. Tokenize & Run Model on Small Sample
Now, we tokenize these conversations and see what the model predicts:

In [ ]:
import torch.nn.functional as F

tokenizer.pad_token = tokenizer.eos_token
# Convert text to tokens
for conv in sample_conversations:
    inputs = tokenizer(
        conv["conversation"],
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()  # Get predicted class

    # Print results
    print(f"Conversation: {conv['conversation'][:100]}...")  # Print first 100 chars
    print(f"Actual Label: {conv['label']}")
    print(f"Predicted Label: {labels[prediction]}")
    print("=" * 50)


# Step 3: Preparing the Dataset for Fine-Tuning
Now, we’ll prepare the main dataset (urgency_data.csv) for fine-tuning by:

1. Loading and inspecting the dataset.
2. Converting text and labels into model-compatible format.
3. Splitting the dataset into train and validation sets.
4. Ensuring efficient memory usage.



# 3.1 Load & Inspect the Dataset

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("./urgency_data.csv")

# Display some samples
print(df.head())


# 3.2 Encode Labels into Numerical Format
Since models require numerical inputs, we map labels to integers

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])

# Store label mapping for future decoding
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Label Mapping:", label_mapping)


# 3.3 Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["conversation"].tolist(),
    df["label_encoded"].tolist(),
    test_size=0.2,
    random_state=42
)

print(f"Training Samples: {len(train_texts)}, Validation Samples: {len(val_texts)}")


# Step 4: Tokenization & Memory Optimization
This step ensures efficient processing by:
1. Tokenizing conversations using the pre-trained model’s tokenizer.
2. Truncating or padding sequences to a fixed length.
3. Using PyTorch Dataset for efficient batch processing.

In [ ]:
# Load tokenizer

# Define a padding token if it's missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding
    model.config.pad_token_id = tokenizer.pad_token_id  # Assign it to the model

# Tokenize training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

print("Sample Tokenized Input:", train_encodings["input_ids"][0][:20])  # First 20 tokens

# 4.2 Create PyTorch Dataset for Efficient Batching

Since Hugging Face models use PyTorch, we need to wrap tokenized data into a dataset.

In [ ]:
import torch

class UrgencyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Convert tokenized data into PyTorch Datasets
train_dataset = UrgencyDataset(train_encodings, train_labels)
val_dataset = UrgencyDataset(val_encodings, val_labels)

print("Dataset Created. Training Samples:", len(train_dataset), "| Validation Samples:", len(val_dataset))


# 4.3 Set Up DataLoaders for Optimized Training

We use DataLoaders to handle batching & parallel processing efficiently.

In [ ]:
from torch.utils.data import DataLoader

# Set batch size (adjust based on RAM availability)
BATCH_SIZE = 1

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

print("Dataloaders Ready! Batch Size:", BATCH_SIZE)


# Step 5: System Resource Check Before Fine-Tuning

We’ll monitor system memory, CPU, and GPU usage to avoid crashes during training.

In [ ]:
import psutil

# Check RAM usage
ram = psutil.virtual_memory()
print(f"Total RAM: {ram.total / 1e9:.2f} GB")
print(f"Available RAM: {ram.available / 1e9:.2f} GB")
print(f"Used RAM: {ram.used / 1e9:.2f} GB ({ram.percent}%)")

# Check disk usage
disk = psutil.disk_usage('/')
print(f"Total Disk Space: {disk.total / 1e9:.2f} GB")
print(f"Available Disk Space: {disk.free / 1e9:.2f} GB")
print(f"Used Disk Space: {disk.used / 1e9:.2f} GB ({disk.percent}%)")

import torch

# Check if GPU is available
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Current GPU Memory Usage: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"Free GPU Memory: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
else:
    print("No GPU found. Running on CPU!")

import os
import gc

# Get process memory usage
pid = os.getpid()
proc = psutil.Process(pid)
print(f"Python Process Memory Usage: {proc.memory_info().rss / 1e9:.2f} GB")

# Run garbage collection
gc.collect()
torch.cuda.empty_cache()
print("Cleared Cache to Free Memory!")



# Step 6: Fine-Tuning DeepSeek Model
6.1 Setup Training Arguments
We'll define our training parameters, including batch size, learning rate, and epochs.

In [ ]:
from transformers import TrainingArguments

model.gradient_checkpointing_enable()  # Saves memory at the cost of extra compute

training_args = TrainingArguments(
    output_dir="./deepseek-finetuned",   # Save model here
    evaluation_strategy="epoch",         # Evaluate after each epoch
    save_strategy="epoch",               # Save model after each epoch
    learning_rate=3e-5,                   # Standard LR for transformers
    per_device_train_batch_size=1,        # Reduce if memory issue
    per_device_eval_batch_size=1,
    num_train_epochs=3,                   # We can adjust this later
    weight_decay=0.01,                     # Regularization
    logging_dir="./logs",                  # Log directory
    logging_steps=10,                       # Log progress
    bf16=True,                              # Use mixed precision for speed
    push_to_hub=False,                       # Disable model upload
    gradient_accumulation_steps=16,
)


# 6.2 Initialize Trainer
We'll use Hugging Face’s Trainer to fine-tune our model

In [ ]:
from transformers import Trainer

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True" # Enable CUDA Memory Expansion

trainer = Trainer(
    model=model,                 # Our DeepSeek model
    args=training_args,
    train_dataset=train_dataset, # Processed training data
    eval_dataset=val_dataset,    # Processed validation data
    tokenizer=tokenizer,
)


# 6.3 Start Fine-Tuning! 🚀
Now, we train the model.

In [ ]:
trainer.train()

## After training, we save the model so we can use it later.

model.save_pretrained("./deepseek-finetuned")
tokenizer.save_pretrained("./deepseek-finetuned")